In [ ]:
import numpy as np 
import os 
import sys 
import glob 
import xarray as xr
import dask
from dask.distributed import Client

path = "/home/b/b382291/git/one_pass"
sys.path.append(path)
os.chdir(path)

from one_pass.convert_time import convert_time
from one_pass.check_request import check_request
from one_pass import util
from one_pass.opa import Opa
!git branch 

### select input and out put directory
outdir = '/work/bb1153/b382220/KOSTRA/Tests/iamser_out/'
# Check if the directory exists, and create it if not
if not os.path.exists(outdir):
    os.makedirs(outdir)
    
indir = "/work/bb1153/b382220/KOSTRA/kostrax/DWData/"
file_list = sorted([file for file in os.listdir(indir) if file.endswith('.nc')])

!hostname

### load data and select variables
ds = xr.open_mfdataset(f'{indir}*.nc') ### load data as dask array
pr = ds['pr'] ### select data variable for total precipitation
da = xr.DataArray(pr)
# have to change the precision, data set has strange time stamps
da['time'] = da.time.astype("datetime64[s]")

* iams
  main
  output_for_bias_corr
  t_digest
  test_package
l40290.lvt.dkrz.de


/tmp/ipykernel_179193/85934964.py:35: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  da['time'] = da.time.astype("datetime64[s]")


: 

extracting a small subset of the data otherwise it's crazy long 

In [2]:
da = da[:,0:10,0:10]

# this is our user requst 

pass_dic = {"stat" : "iams",
"percentile_list" : None,
"thresh_exceed" : None,
"stat_freq": "annually",
"output_freq": "annually",
"time_step": 60,
"variable": "pr",
"save": True,
"checkpoint": True,
"checkpoint_filepath": "/scratch/b/b382291/data/",
"out_filepath": "/scratch/b/b382291/data"}

start = 0
stop =  365*24 # end of 2071
step = 170 # don't go above this for now 

opa_stat = Opa(pass_dic)

for i in range(start, stop, step): 

    data = da.isel(time=slice(i,i+step)) # extract moving window 'simulating streaming'
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

    dm = opa_stat.compute(data)

start_date: 2071-01-01T00:30:00.000000000 stop_date: 2071-01-08T01:30:00.000000000
start_date: 2071-01-08T02:30:00.000000000 stop_date: 2071-01-15T03:30:00.000000000
start_date: 2071-01-15T04:30:00.000000000 stop_date: 2071-01-22T05:30:00.000000000
start_date: 2071-01-22T06:30:00.000000000 stop_date: 2071-01-29T07:30:00.000000000
start_date: 2071-01-29T08:30:00.000000000 stop_date: 2071-02-05T09:30:00.000000000
start_date: 2071-02-05T10:30:00.000000000 stop_date: 2071-02-12T11:30:00.000000000
start_date: 2071-02-12T12:30:00.000000000 stop_date: 2071-02-19T13:30:00.000000000
start_date: 2071-02-19T14:30:00.000000000 stop_date: 2071-02-26T15:30:00.000000000
start_date: 2071-02-26T16:30:00.000000000 stop_date: 2071-03-05T17:30:00.000000000
start_date: 2071-03-05T18:30:00.000000000 stop_date: 2071-03-12T19:30:00.000000000
start_date: 2071-03-12T20:30:00.000000000 stop_date: 2071-03-19T21:30:00.000000000
start_date: 2071-03-19T22:30:00.000000000 stop_date: 2071-03-26T23:30:00.000000000
star

In [9]:
opa_stat.n_data

8760

comparison with two pass 

In [3]:
### all functions
############################################################## to select available duration depending on the input data 
############################################################## time_unit(str) = 'minutes' or 'hours'; time_step(int) = e.g. 1 for hourly, 10 for ten minutes

#-- automatically check timestep of files and rescale
#-- remove durations that are lower than timestep and round
#-- check if duration is a fraction of multiple timesteps
def duration_pick(time_unit, time_step):
    durations  = [5,10,15,20,30,45,60,90,120,180,240,360,540,720,1080,1440,2880,4320,5760,7200,8640,10080] ### list of all duration windows that may be passed on
    
    if time_unit != 'minutes': ### format time unit, when not minutes
        durations = [d/60 for d in durations]

    durations = [d for d in durations if d >= time_step] ### loop over duration windows that fall in the range of the time step
    for d in durations:
        if d%time_step != 0: ### only select natural numbers
            durations.remove(d)  
    durations = list(map(int, durations)) ### create list of only intergers
    return durations

############################################################## compute intensity annual maxima series (iams) for single duration window
############################################################## 

def dm_roller(data, duration, year): ###(ds.'precipitation', list()selected_duration windows, str()year)
    
    
    ### compute rolling sum & select maximum
    rolling_sum = data.rolling(time=duration, center=True).sum() ### compute rolling sum for overlapping duration windows
    rolling_sum_max = rolling_sum.max(dim='time', skipna=True)  ### use .max() to select sum with maximum value
    
    ### add coordinates to the array
    if time_unit != 'minutes':
        rolling_sum_max = rolling_sum_max.expand_dims(duration = ([duration*60])) ### if not minutes (hourly) transform windows back to minutes 
    else: 
        rolling_sum_max = rolling_sum_max.expand_dims(duration = ([duration])) ### add coordinate duration to data array
        
    #year= np.unique(data.time.dt.year)    
    rolling_sum_max = rolling_sum_max.expand_dims(year = ([year])) ### coordinate year
    
    return rolling_sum_max

############################################################## For data of several years: compute intensity annual maxima series (iams) for single duration window
##############################################################

def iamser(data, years, durations):###(ds.'precipitation', list()selected_duration windows, list()years)
    with dask.config.set(**{'array.slicing.split_large_chunks': True}): ### set dask to slice large chunks 'False' also works dunno
        # Your array indexing/slicing operations
        all_years_iams = [] ### create list to gather yearly arrays
        data_grouped = data.groupby('time.year') ### group data into years
        #years= list(np.unique(pr.time.dt.year)) ### create list with all years
        for y in years: ### loop over years in year list
            print(y)
            #group_with_time = group.assign_coords(time=group['time'])
            dmax_list = [] ### list to add each duartion sum max
            for durs in durations: ### loop over selected durations
                dmax = dm_roller(data_grouped[y], durs, y) ### call iamser function for one year
                dmax_list.append(dmax) ### append iams output to iams_list
            iamsfile = xr.concat(dmax_list, dim='duration') ### concat all durations into one array
            all_years_iams.append (iamsfile) ### add entire year to list 

        #iams = xr.concat(all_years_iams, dim='year') ### final concat, all years with all durations as one array
        #return iams 
        return all_years_iams ### reuturn list, easier to save data as .nc year for year


In [4]:
### pick possible duration windows
time_step = 1 ### define data temporal fruequency
time_unit = 'hour' ### define data time unit
sel_durs = duration_pick('hours',1) ### call sel_durs to select all possible windows within given time frquency & unit

### load data and select variables
ds = xr.open_mfdataset(f'{indir}*.nc') ### load data as dask array
#ds = xr.open_mfdataset(f'{indir}{file_list[4]}')
pr = ds['pr'] ### select data variable for total precipitationpr
pr= pr[:,0:10,0:10]
#pr = ds['tp']
years= [2071] # list(np.unique(pr.time.dt.year)) ### create list for all years in the given data set 

### run functions to create intensity annual  maxima series
iams = iamser(pr, years, sel_durs)

2071


In [5]:
iams

[<xarray.DataArray 'pr' (year: 1, duration: 15, rlat: 10, rlon: 10)>
 dask.array<concatenate, shape=(1, 15, 10, 10), dtype=float32, chunksize=(1, 1, 10, 10), chunktype=numpy.ndarray>
 Coordinates:
   * year      (year) int64 2071
   * duration  (duration) int64 60 120 180 240 360 ... 4320 5760 7200 8640 10080
   * rlon      (rlon) float64 -10.29 -10.27 -10.24 -10.21 ... -10.1 -10.07 -10.05
   * rlat      (rlat) float64 -4.496 -4.469 -4.441 ... -4.304 -4.276 -4.249
     lat       (rlat, rlon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
     lon       (rlat, rlon) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>]

In [6]:
iams[0].values[0,6,5,2]

In [ ]:
dm.pr.values[0,6,5,2]

0.04076696652381135